In [4]:
%load_ext autoreload
%autoreload 2

# Capturing a video to a file with the Pi

In [5]:
%%writefile 2017-05-19_capture_videos.py
#! /usr/bin/env python3
# -*- coding: utf8 -*-

import picamera

class Rec_video():
    def __init__(self):
        cam = picamera.PiCamera()
        cam.resolution = (640,480) #Defining the camera resolution        
        cam.start_recording(output='video_picam2', format='h264') #Start recording in 'filename.filetype'
        cam.wait_recording(5) #Better integrated time.wait() that allow to continually check for errors
        cam.stop_recording() #Stop recording

        print('Recording end') #Control line
        
rec = Rec_video() #Calling the Rec_video class

Overwriting 2017-05-19_capture_videos.py


In [39]:
!git add 2017-05-19_capture_videos.py
!git commit -m' updating *_capture_videos.py '; git push
!ssh pi@10.164.7.213 "cd ~/openRetina ; git pull; cd ~/INT-internship ; git pull"

On branch master
Your branch is up-to-date with 'origin/master'.
nothing to commit, working tree clean
Everything up-to-date
Already up-to-date.
From https://github.com/pierrealbiges/INT-internship
   e24d0b1..18a5418  master     -> origin/master
Updating b975f4a..18a5418
Fast-forward
 .gitignore                                         |  3 +-
 ...-05-17_capture_videos_with_openretina_server.py | 16 ++--
 ...2017-05-18_capture_videos_with_openretina.ipynb | 94 +++++++++++++---------
 2017-05-18_capture_videos_with_openretina.py       | 16 ++++
 ...-18_capture_videos_with_openretina_server.ipynb | 84 ++++++++++++-------
 ...-05-18_capture_videos_with_openretina_server.py | 39 +++++++++
 W4_Progress_and_notes.ipynb                        | 16 +++-
 7 files changed, 189 insertions(+), 79 deletions(-)
 rename 2017-05-17_capture_videos_with_openretina.ipynb => 2017-05-18_capture_videos_with_openretina.ipynb (62%)
 create mode 100644 2017-05-18_capture_videos_with_openretina.py
 rename 2017-

In [6]:
!ssh pi@10.164.7.213 "cd ~/INT-internship ; python3 2017-05-19_capture_videos.py"

Recording end


# Sending the video file to the computer

In [1]:
%%writefile 2017-05-19_capture_videos_client.py
""" Pi side """

import zmq
CHUNK_SIZE = 250000

try:
    context = zmq.Context()
    router = context.socket(zmq.ROUTER)
    
    router.sndhwm = router.rcvhwm = CHUNK_SIZE
    router.hwm = CHUNK_SIZE
    router.bind("tcp://*:5555")
    print("Connected to computer")
    
    file = open("video_picam2","rb")
    print('File opened')
    while True:
        identity, command = router.recv_multipart()
        
        assert command == b"fetch"
        print('Command received')
        
        while True:
            data = file.read(CHUNK_SIZE)
            router.send_multipart([identity, data])
            print('Sending data...')
            if not data:
                print('\nNo more data')
                break
        if not data: break

finally:
    router.close()
    context.term()
    print("\nConnection closed")

Overwriting 2017-05-19_capture_videos_client.py


In [2]:
!git add 2017-05-19_capture_videos_client.py
print('PUSHING')
!git commit -m' updating *_capture_videos_client.py '; git push
print('\nPULLING')
!ssh pi@10.164.7.213 "cd ~/openRetina ; git pull; cd ~/INT-internship ; git pull"

PUSHING
[master a0fca1f]  updating *_capture_videos_client.py
 1 file changed, 1 insertion(+), 1 deletion(-)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 326 bytes | 0 bytes/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:pierrealbiges/INT-internship.git
   4b5b86c..a0fca1f  master -> master

PULLING
From https://github.com/laurentperrinet/openRetina
   ea4c762..1059322  master     -> origin/master
Updating ea4c762..1059322
Fast-forward
 src/__init__.py    |   4 +-
 src/eventRetina.py |  52 ++++++++++
 src/openRetina.py  | 277 ++++++++++++++++++++++++++++++++---------------------
 3 files changed, 225 insertions(+), 108 deletions(-)
 create mode 100644 src/eventRetina.py
From https://github.com/pierrealbiges/INT-internship
   1ebfb23..a0fca1f  master     -> origin/master
Updating 1ebfb23..a0fca1f
Fast-forward
 .git

In [1]:
!ssh pi@10.164.7.213 "cd ~/INT-internship/ ; python3 2017-05-19_capture_videos_client.py"